In [ ]:
pip install keras-tcn

     |████████████████████████████████| 378kB 8.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
import tensorflow.keras as keras
import tensorflow as tf
from keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import History, ModelCheckpoint, TensorBoard
import numpy as np
import tcn
from tcn import TCN, tcn_full_summary
import pickle
from urllib.request import urlopen
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tcn import compiled_tcn
import os

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#with open('/content/drive/My Drive/wavenet-test/features.pickle', 'rb') as handle:
#    features = pickle.load(handle)
    
#with open('/content/drive/My Drive/wavenet-test/labels.pickle', 'rb') as handle:
#  labels = pickle.load(handle)

#!cp '/content/drive/My Drive/wavenet-test/US8K.tar' .

In [ ]:
!nvidia-smi

Sat Nov 28 18:32:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Utils

def create_new_model(depth, filters, kernel_size):
    input_layer = keras.layers.Input(shape=(341, 513, 2))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", kernel_initializer="he_normal", name="conv1", padding="same")(input_layer)
    # Stride of (1, 2) -> stride of 2 in feature dimension, reducing feature dimensionality per timestep
    conv2 = keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1,2), activation="relu", kernel_initializer="he_normal", padding="same")(conv1)
    
    # Slicing 1D Flattening!
    feature_distributor = keras.layers.TimeDistributed(keras.layers.Flatten())(conv2)
    
    # TCN Unit
    tcn1 = tcn.TCN(return_sequences=False, kernel_size=(kernel_size), kernel_initializer="he_normal", nb_filters=filters, 
                   dilations=[2 ** i for i in range(depth)], nb_stacks=2, use_batch_norm=True,
                   dropout_rate=0.5, name="tcn1", use_skip_connections=True, padding="same")(feature_distributor)
    tcn1 = keras.layers.BatchNormalization()(tcn1)
 
    last_layer = keras.layers.Dense(20, activation="softmax")(tcn1)
    
    output_layer = keras.layers.Dense(10, activation="softmax")(last_layer)

    model = keras.Model(inputs=input_layer, outputs=output_layer)

    optimizer = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=["categorical_accuracy"])
    return model



def load_model(depth, filters, kernel_size, save_dir="./"):
  model = create_new_model(depth, filters, kernel_size)
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  # model.load here
  if os.path.isfile(saved_model):
    model.load_weights(saved_model)
  else:
    print("NO MODEL ON DISK, USING BRAND NEW ONE")
  return model

def create_callbacks(save_dir="./"):
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  saved_hist  = save_dir + 'wavenet_classifier_training_history.csv'
  checkpointer = ModelCheckpoint(filepath=saved_model, monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
  #tensorboard  = TensorBoard(log_dir=save_dir)
  history_cb = tf.keras.callbacks.CSVLogger(saved_hist, separator=",", append=True)

  return [checkpointer, history_cb] # add tensorboard if needed

In [ ]:
all_X = []
all_Y = []
fulldatasetpath = './'
metadata = pd.read_csv('http://76.213.149.126:8081/metadata/UrbanSound8K.csv')
print(metadata)
for index, row in metadata.iterrows():
    
    file_name = 'fold'+str(row["fold"])+'/' + str(row["slice_file_name"])
    all_X.append(file_name.replace(".wav", "_x2.pkl"))
    all_Y.append(file_name.replace(".wav", "_y2.pkl"))

import requests
from multiprocessing.pool import ThreadPool
 
def download_url(url):
  #print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  path = url.split("/")
  path = path[-2] + "/" + path[-1]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    if not os.path.exists(os.path.dirname(path)):
      try:
          os.makedirs(os.path.dirname(path))
      except OSError as exc: # Guard against race condition
          if exc.errno != errno.EEXIST:
              raise
    with open(path, 'wb') as f:
      for data in r:
        f.write(data)
  return url

from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(all_X, all_Y, test_size=0.2, random_state = 42)

def file_to_arr(t: tf.Tensor):
  x = []
  y = []

  urls = []

  for a in t:
    a = np.array(a)
    # check whether we have them
    # if not, schedule download
    if not os.path.exists(a[0]):
      urls.append("http://76.213.149.126:8081/audio/" + a[0].decode('UTF-8'))
    if not os.path.exists(a[1]):
      urls.append("http://76.213.149.126:8081/audio/" + a[1].decode('UTF-8'))
  
  if (len(urls) > 0):
    ThreadPool(8).map(download_url, urls)

  for a in t:
    a = np.array(a)
    #print(a[0])
    try:
      x.append(pickle.load(open(a[0], 'rb')))
      y.append(pickle.load(open(a[1], 'rb')))
    except:
      os.remove(a[0])
      os.remove(a[1])
      return file_to_arr(t)

  x = np.array(x)
  x = np.swapaxes(x, 1, 2)
  y = np.array(y).astype(int)
  res = (x,y)
  #print (res)
  return (res)

print((list(zip(x_train, y_train))))
train_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_train, y_train)))
train_ds = train_ds.shuffle(len(train_ds),seed=42)
train_ds = train_ds.batch(8)
train_ds = train_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
train_ds = train_ds.prefetch(128)

test_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_test, y_test)))
test_ds = test_ds.batch(8)
test_ds = test_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
test_ds = test_ds.prefetch(128)

         slice_file_name    fsID       start  ...  fold  classID             class
0       100032-3-0-0.wav  100032    0.000000  ...     5        3          dog_bark
1     100263-2-0-117.wav  100263   58.500000  ...     5        2  children_playing
2     100263-2-0-121.wav  100263   60.500000  ...     5        2  children_playing
3     100263-2-0-126.wav  100263   63.000000  ...     5        2  children_playing
4     100263-2-0-137.wav  100263   68.500000  ...     5        2  children_playing
...                  ...     ...         ...  ...   ...      ...               ...
8727     99812-1-2-0.wav   99812  159.522205  ...     7        1          car_horn
8728     99812-1-3-0.wav   99812  181.142431  ...     7        1          car_horn
8729     99812-1-4-0.wav   99812  242.691902  ...     7        1          car_horn
8730     99812-1-5-0.wav   99812  253.209850  ...     7        1          car_horn
8731     99812-1-6-0.wav   99812  332.289233  ...     7        1          car_horn

[87

In [ ]:
#model = create_new_model(7, 20, 6)
model = load_model(7, 20, 6, "./")

In [ ]:
#callbacks = create_callbacks("./drive/My Drive/wavenet-test/new2/7_20_3_1stack_noskip/")
callbacks = create_callbacks("./")

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 341, 513, 2)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 341, 513, 32)      608       
_________________________________________________________________
conv2d (Conv2D)              (None, 341, 257, 32)      9248      
_________________________________________________________________
time_distributed (TimeDistri (None, 341, 8224)         0         
_________________________________________________________________
tcn1 (TCN)                   (None, 20)                1218980   
_________________________________________________________________
batch_normalization (BatchNo (None, 20)                80        
_________________________________________________________________
dense (Dense)                (None, 20)               

In [ ]:
!mkdir ./drive/My\ Drive/wavenet-test/new2/7_20_3_1stack_noskip/

In [ ]:
model.fit(train_ds, validation_data=test_ds, callbacks=callbacks, epochs=300)

Epoch 1/300
975/975 [==============================] - ETA: 0s - loss: 1.2369 - categorical_accuracy: 0.5535
Epoch 00001: val_categorical_accuracy improved from -inf to 0.51709, saving model to ./saved_wavenet_clasifier.h5
975/975 [==============================] - 307s 315ms/step - loss: 1.2369 - categorical_accuracy: 0.5535 - val_loss: 1.4073 - val_categorical_accuracy: 0.5171
Epoch 2/300
975/975 [==============================] - ETA: 0s - loss: 1.1795 - categorical_accuracy: 0.5713
Epoch 00002: val_categorical_accuracy did not improve from 0.51709
975/975 [==============================] - 318s 326ms/step - loss: 1.1795 - categorical_accuracy: 0.5713 - val_loss: 1.5024 - val_categorical_accuracy: 0.5128
Epoch 3/300
975/975 [==============================] - ETA: 0s - loss: 1.1410 - categorical_accuracy: 0.5848
Epoch 00003: val_categorical_accuracy did not improve from 0.51709
975/975 [==============================] - 315s 323ms/step - loss: 1.1410 - categorical_accuracy: 0.5848 - 

In [ ]:
!rm -rf fold*

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.0015, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=["categorical_accuracy"])

In [ ]:
t = iter(train_ds)

In [ ]:
g = next(t)

In [ ]:
g

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'fold10/93567-8-0-16_x2.pkl', b'fold10/93567-8-0-16_y2.pkl'],
      dtype=object)>

In [ ]:
[2 ** i for i in range(15)]

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(all_X, all_Y, test_size=0.2, random_state = 42)

x_train

['fold10/93567-8-0-16_x2.pkl',
 'fold2/96920-9-0-8_x2.pkl',
 'fold5/156869-8-0-3_x2.pkl',
 'fold6/184805-0-0-101_x2.pkl',
 'fold6/34643-4-1-1_x2.pkl',
 'fold9/184449-2-0-13_x2.pkl',
 'fold8/125678-7-4-1_x2.pkl',
 'fold5/72259-1-7-13_x2.pkl',
 'fold6/128465-1-0-0_x2.pkl',
 'fold7/177537-7-1-3_x2.pkl',
 'fold8/177726-0-0-13_x2.pkl',
 'fold10/74364-8-1-21_x2.pkl',
 'fold9/155044-9-0-3_x2.pkl',
 'fold3/177742-0-0-99_x2.pkl',
 'fold1/78360-4-0-22_x2.pkl',
 'fold7/104625-4-0-45_x2.pkl',
 'fold5/100852-0-0-17_x2.pkl',
 'fold10/27070-2-0-8_x2.pkl',
 'fold5/65381-3-0-7_x2.pkl',
 'fold5/178686-0-0-2_x2.pkl',
 'fold6/167701-4-0-0_x2.pkl',
 'fold2/165775-7-1-0_x2.pkl',
 'fold2/102871-8-0-7_x2.pkl',
 'fold8/145206-6-1-0_x2.pkl',
 'fold5/148841-6-2-0_x2.pkl',
 'fold10/85574-3-0-0_x2.pkl',
 'fold6/63724-0-0-7_x2.pkl',
 'fold5/148841-6-0-0_x2.pkl',
 'fold10/74364-8-1-7_x2.pkl',
 'fold1/180937-7-3-14_x2.pkl',
 'fold4/162435-6-2-0_x2.pkl',
 'fold10/187110-2-0-30_x2.pkl',
 'fold2/159750-8-0-7_x2.pkl',
 '

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(all_X, all_Y, test_size=0.2, random_state = 42)

x_train = []
x_test = []
y_train = []
y_test = []

for x in all_X:
  if x[0:5] != "fold4":
    x_train.append(x)
  else:
    x_test.append(x)


for y in all_Y:
  if y[0:5] != "fold4":
    y_train.append(y)
  else:
    y_test.append(y)

x_train

['fold5/100032-3-0-0_x2.pkl',
 'fold5/100263-2-0-117_x2.pkl',
 'fold5/100263-2-0-121_x2.pkl',
 'fold5/100263-2-0-126_x2.pkl',
 'fold5/100263-2-0-137_x2.pkl',
 'fold5/100263-2-0-143_x2.pkl',
 'fold5/100263-2-0-161_x2.pkl',
 'fold5/100263-2-0-3_x2.pkl',
 'fold5/100263-2-0-36_x2.pkl',
 'fold10/100648-1-0-0_x2.pkl',
 'fold10/100648-1-1-0_x2.pkl',
 'fold10/100648-1-2-0_x2.pkl',
 'fold10/100648-1-3-0_x2.pkl',
 'fold10/100648-1-4-0_x2.pkl',
 'fold2/100652-3-0-0_x2.pkl',
 'fold2/100652-3-0-1_x2.pkl',
 'fold2/100652-3-0-2_x2.pkl',
 'fold2/100652-3-0-3_x2.pkl',
 'fold10/100795-3-0-0_x2.pkl',
 'fold10/100795-3-1-0_x2.pkl',
 'fold10/100795-3-1-1_x2.pkl',
 'fold10/100795-3-1-2_x2.pkl',
 'fold5/100852-0-0-0_x2.pkl',
 'fold5/100852-0-0-1_x2.pkl',
 'fold5/100852-0-0-10_x2.pkl',
 'fold5/100852-0-0-11_x2.pkl',
 'fold5/100852-0-0-12_x2.pkl',
 'fold5/100852-0-0-13_x2.pkl',
 'fold5/100852-0-0-14_x2.pkl',
 'fold5/100852-0-0-15_x2.pkl',
 'fold5/100852-0-0-16_x2.pkl',
 'fold5/100852-0-0-17_x2.pkl',
 'fold5/10

In [ ]:
occurances = {
    "0": 0,
    "1": 0,
    "2": 0,
    "3": 0,
    "4": 0,
    "5": 0,
    "6": 0,
    "7": 0,
    "8": 0,
    "9": 0
}

for x in x_test:
  print(x.split("-")[1])
  occurances[x.split("-")[1]] += 1

occurances

3
9
9
9
9
9
9
3
3
3
7
7
7
7
8
9
9
9
9
9
9
3
3
3
3
3
1
3
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
9
9
9
9
9
9
7
7
7
7
7
7
7
7
7
9
9
9
9
9
9
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
2
2
2
2
2
2
2
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
8
8
8
8
8
6
3
3
3
3
6
8
8
8
8
8
8
8
8
8
8
8
6
3
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
1
1
1
1
6
6
6
6
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
2
2
2
2
2
2
2
2
1
1
1
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
9
9
9
9
9
9
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
2
2
2
2
2
2
2
2
1
5
5
5
2
2
2
2
2
2
2
2
4
1
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2


{'0': 100,
 '1': 59,
 '2': 100,
 '3': 100,
 '4': 100,
 '5': 107,
 '6': 38,
 '7': 120,
 '8': 166,
 '9': 100}

In [ ]:
occurances

{'0': 900,
 '1': 370,
 '2': 900,
 '3': 900,
 '4': 900,
 '5': 893,
 '6': 336,
 '7': 880,
 '8': 763,
 '9': 900}

In [ ]:
print(pickle.load(open('fold5/102547-3-0-7_x2.pkl', 'rb')))

FileNotFoundError: ignored

In [ ]:
!rm -rf fold5/102547-3-0-7_x2.pkl

In [ ]:
a[0]

NameError: ignored